In [ ]:
import gurobipy as gp
from gurobipy import GRB


# Initialise Model
m = gp.Model("8-3-2")


# Initialise Decision Variables using a list of indices from 1 to 336
indices = list(range(0, 336))
indices_2 = list(range(0, 7))

x = m.addVars(indices, vtype=GRB.BINARY, name='x')
c = m.addVars(indices, lb = 0, vtype=GRB.CONTINUOUS, name='c')
z = m.addVars(indices, vtype=GRB.INTEGER, name='z')
y = m.addVars(indices, lb = 0, vtype=GRB.CONTINUOUS, name='y')
beta = m.addVars(indices_2, lb = 0, vtype=GRB.INTEGER, name='Beta')  # Beta_0, ... Beta_6


# Set bjective function
m.setObjective(y.sum())


# Define the alpha matrix
import itertools
# Create all unique pairs for the face-up cards (left and right)
card_values = range(1, 9)  # Card values from 1 to 8
face_up_pairs = list(itertools.permutations(card_values, 2))

# Generate alpha matrix
alpha_matrix = []
condensed_matrix = []


for left, right in face_up_pairs:
    for hidden in card_values:
        if hidden not in (left, right):  # The hidden card must be different
            alpha_i1 = left
            alpha_i2 = right
            alpha_i3 = 1 if left > right else 0
            alpha_i4 = 1 if hidden > left else 0
            alpha_i5 = 1 if hidden > right else 0
            alpha_j6 = hidden
            alpha_matrix.append([alpha_i1, alpha_i2, alpha_i3, alpha_i4, alpha_i5, alpha_j6])

            sorted_row = sorted([alpha_i1, alpha_i2,alpha_j6])
            condensed_matrix.append(sorted_row)


# alpha_matrix
# condensed_matrix



# This is to for ease of coding out the matching constraints (see the variable 'new_matrix')
from collections import defaultdict

# Initialize a dictionary to store the rows grouped by their elements
grouped_rows = defaultdict(list)

# Iterate through the condensed matrix
for index, row in enumerate(condensed_matrix):
    # Convert the row to a tuple to make it hashable and usable as a dictionary key
    key = tuple(row)
    # Append the row index to the list corresponding to the key
    grouped_rows[key].append(index)


# Define the size of the new matrix
num_rows = len(grouped_rows)
num_cols = 6  # Assuming all rows have the same number of columns

# Initialize a new matrix with zeros
new_matrix = [[0] * num_cols for _ in range(num_rows)]

# Iterate through the grouped rows
for row_index, indices in enumerate(grouped_rows.values()):
    # Fill the new matrix with indices
    for col_index, index in enumerate(indices):
        new_matrix[row_index][col_index] = index

# Print the new matrix
for row in new_matrix:
    print(row)


# Create constraints
P = 7

for i in range(336):
    m.addConstr(y[i] + (1 - x[i]) * P >= c[i])  # change

for i in range(56):
    m.addConstr(gp.quicksum(x[new_matrix[i][j]] for j in range(6)) == 1)

for i in range(56):
    m.addConstr(gp.quicksum(x[6*i + j] for j in range(6)) <= 1, 'x_sum_{}'.format(i))

for i in range(336):
    m.addConstr(c[i] == beta[0] + gp.quicksum(beta[k+1] * alpha_matrix[i][k] for k in range(0, 6)) - z[i] * P)



# Additional Cuts
m.addConstr(beta.sum() >= 1)

for i in range(7):
    m.addConstr(beta[i] <= P-1)  # P = 7 for (8-3-2) model

# Additional cuts
m.addConstr(beta[1] == beta[2])

for i in range(6):
    m.addConstr(beta[i+1] >= 1)

for i in range(336):
   m.addConstr(y[i] + (1 - x[i]) <= 1)  


# Solve the Model
m.setParam('TimeLimit', 5*60) # need around 3 min 30 sec to find the optimal obj = 8
m.optimize()


# Print the coefficients Beta
for i in range(7):
    print('beta[{}]: {}'.format(i, beta[i].X))


# Print the values of the 'c' variables
# for i in range(336):
#     print('c[{}]: {}'.format(i, c[i].X))


# Initialize a counter for zero_cost links
zero_count = 0

for i in range(336):
     if c[i].X == 0.0:
        zero_count += 1
         
print("There are", zero_count, "zero-cost links")


# Print the matching links and the corresponding costs
matching_with_zero_cost = 0
for i in range(336):
    if (x[i].X == 1):
        print('x[{}]: {}'.format(i, x[i].X), 'c[{}]: {}'.format(i, c[i].X))
        if (c[i].X == 0):
            matching_with_zero_cost += 1

print("Out of the 56 matching links, there are", matching_with_zero_cost, "matching links with cost zero")